In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('sequence_data_anonymized.csv')
df

# Select duplicated rows and keep largest motif

In [ ]:
duplicates_df = df[df.duplicated(subset='seq', keep=False)].copy()
duplicates_df

In [ ]:
duplicates_df['insert_motif_len'] = duplicates_df['insert_motif'].str.len()
grouped = duplicates_df.groupby('seq')
mask = grouped['insert_motif_len'].transform(lambda x: x == x.max()).astype(bool)
selection_df = duplicates_df.loc[mask].reset_index(drop=True)
selection_df.drop(columns='insert_motif_len', inplace=True)
selection_df

## Ensure no more duplicates

In [ ]:
len(selection_df) == len(selection_df['seq'].unique())

# Select non-duplicated entries

In [ ]:
df_unique = df.iloc[~df.index.isin(duplicates_df.index)].copy()
df_unique

# Combine unique sequences with selection from duplicates

In [ ]:
fulldf = pd.concat([df_unique, selection_df], ignore_index=True).reset_index(drop=True)
fulldf = fulldf.sort_values(by='seq_id_anonymized').reset_index(drop=True)
fulldf

## Export

In [ ]:
fulldf.to_csv('sequence_data_anonymized_no_duplicates.csv', index=False)

with open("design_files_noduplicates.fasta", "w") as f:
    for row in fulldf.iterrows():
        f.write(">" + row[1]['seq_id_anonymized'] + "\n" + row[1]['seq'] + "\n")